In [1]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# 五粮液 000858
# 格力电器 000651
# 美的集团 000333

In [3]:
raw_df = pd.read_csv('../input/000858_CONSOLIDATED.csv')
raw_df.columns = [col.lower() for col in raw_df.columns]
raw_df = raw_df.sort_values(by='report_date', ascending=False)
raw_df.head()

,security_code,security_name_abbr,report_date,total_assets,fixed_asset,monetaryfunds,monetaryfunds_yoy,accounts_rece,accounts_rece_yoy,inventory,inventory_yoy,total_liabilities,accounts_payable,accounts_payable_yoy,advance_receivables,advance_receivables_yoy,total_equity,total_equity_yoy,total_assets_yoy,total_liab_yoy,current_ratio,debt_asset_ratio,parent_netprofit,total_revenue,total_operate_cost,total_expense_yoy,toi_ratio,operate_cost,operate_expense,operate_expense_yoy,sale_expense,manage_expense,finance_expense,operate_profit,total_profit,income_tax,operate_income,interest_ni,interest_ni_ratio,netcash_operate,netcash_operate_yoy,sales_services_cash,sales_services_cash_yoy,pay_staff_cash,pay_staff_cash_yoy,netcash_invest,netcash_invest_yoy,receive_invest_income,receive_invest_income_yoy,construct_long_asset,construct_long_asset_yoy,netcash_finance,netcash_finance_yoy,basic_eps,deduct_basic_eps,weightavg_roe,operate_income_yoy,netprofit_yoy,book_value_per_share,netcash_operate_per_share,gross_margin,operate_income_mom,netprofit_mom,qdate
1,858,五粮液,2023-09-30 00:00:00,1.518034e+11,5.250182e+09,9.814771e+10,23.727348,1.406224e+08,10.382347,1.562720e+10,4.450182,2.716341e+10,7.453945e+09,-2.070087,21076698.67,27.356450,1.246400e+11,13.631378,12.710345,8.668737,500.658448,17.893812,2.283293e+10,6.253639e+10,3.115269e+10,11.170684,12.113192,1.509677e+10,1.509677e+10,12.642755,6.227271e+09,2.403022e+09,-1.812648e+09,3.168408e+10,3.162806e+10,7.794083e+09,NaN,NaN,NaN,2.238875e+10,462.548766,6.681847e+10,1380.461593,6.169873e+09,127.468819,-2.418850e+09,-49.973149,23638368.0,0.488366,2.443148e+09,50.475141,-1.512960e+10,-312.575618,5.882,NaN,19.20,12.113192,14.24,31.476796,5.767905,75.859228,18.5312,28.9590,2023Q3
2,858,五粮液,2023-06-30 00:00:00,1.505734e+11,5.255604e+09,9.255730e+10,22.882954,7.857872e+07,-36.097269,1.609827e+10,12.100769,3.077844e+10,7.615342e+09,16.409137,18726957.90,58.972136,1.197949e+11,13.564457,15.544903,23.958646,441.564437,20.440828,1.703671e+10,4.550638e+10,2.202668e+10,8.798358,10.392431,1.056796e+10,1.056796e+10,11.074424,4.320446e+09,1.756899e+09,-1.250071e+09,2.371557e+10,2.367227e+10,5.879733e+09,NaN,NaN,NaN,1.132982e+10,1377.414072,4.305169e+10,5233.975750,4.794521e+09,582.890232,-1.344449e+09,-163.450321,NaN,NaN,1.345044e+09,163.522637,-1.080791e+10,-1313.963751,4.389,4.364,14.19,10.392431,12.83,29.983544,2.918847,76.776972,-53.8598,-64.1637,2023Q2
4,858,五粮液,2023-03-31 00:00:00,1.604211e+11,5.254760e+09,1.011283e+11,28.742068,8.845112e+07,-50.377978,1.606515e+10,16.024157,3.062763e+10,7.492576e+09,19.232651,16420891.54,38.333959,1.297935e+11,15.119885,16.412182,22.226754,476.174727,19.092019,1.254209e+10,3.113886e+10,1.385794e+10,11.752605,13.034273,6.727661e+09,6.727661e+09,13.104688,2.112310e+09,1.097441e+09,-6.144575e+08,1.747222e+10,1.742402e+10,4.317548e+09,NaN,NaN,NaN,9.535618e+09,114.626692,2.863684e+10,344.240571,3.223645e+09,38.751107,-1.104264e+09,-13.274243,NaN,NaN,1.104204e+09,13.273524,-1.125082e+08,-1.352450,3.231,NaN,10.43,13.034273,15.89,32.607616,2.456615,78.394648,71.1966,87.1583,2023Q1
0,858,五粮液,2022-12-31 00:00:00,1.528119e+11,5.312971e+09,9.235843e+10,12.172654,3.568694e+07,-44.406902,1.598066e+10,14.024834,3.612490e+10,7.246803e+09,34.111601,16160671.49,47.311796,1.166870e+11,15.085176,12.604198,5.263412,384.700951,23.640104,2.669066e+10,7.396864e+10,3.704902e+10,9.103559,11.719828,1.817843e+10,1.817843e+10,11.395749,6.844237e+09,3.068119e+09,-2.026351e+09,3.717442e+10,3.710352e+10,9.132889e+09,NaN,NaN,NaN,2.443114e+10,254.242414,8.177058e+10,850.944883,7.879083e+09,81.993609,-1.716455e+09,-17.862270,23038080.0,0.239746,1.780535e+09,18.529121,-1.310543e+10,-136.381562,6.876,6.869,25.28,11.719828,14.17,29.376459,6.294076,75.424145,24.9472,37.0303,2022Q4
5,858,五粮液,2022-09-30 00:00:00,1.346845e+11,5.412998e+09,7.932580e+10,8.433757,1.273957e+08,9.430902,1.496139e+10,10.293854,2.499653e+10,7.611510e+09,52.590381,16549376.75,32.474495,1.096880e+11,15.394209,13.886084,7.708982,487.8

In [6]:
raw_df['quarter'] = raw_df['qdate'].str.extract(r'(\d)$').fillna(0).astype(int)

# ROE
# 1. roe
# 2. net margin
# 3. asset turnover
# 4. financial leverage
raw_df['net_margin'] = raw_df['parent_netprofit'] / raw_df['total_revenue']
raw_df['total_asset_turnover'] = raw_df['total_revenue'] / raw_df['total_assets'] / raw_df['quarter'] * 4
raw_df['financial_leverage'] = raw_df['total_assets'] / raw_df['total_liabilities']

# quality
# 1. operate cash inflow to revenue
# 2. operate net cash flow to net profit
# 3. free cash flow
raw_df['cash_to_revenue'] = raw_df['sales_services_cash'] / raw_df['total_revenue']
raw_df['cfo_to_netprofit'] = raw_df['netcash_operate'] / raw_df['operate_profit']
raw_df['free_cash_flow'] = raw_df['sales_services_cash'] - raw_df['construct_long_asset']

# activity ratios
raw_df['days_sales_receivables'] = raw_df['accounts_rece'] / raw_df['total_revenue'] * 365 / raw_df['quarter'] * 4
raw_df['days_inventory_on_hands'] = raw_df['inventory'] / raw_df['operate_cost'] * 365 / raw_df['quarter'] * 4
raw_df['days_sales_payables'] = raw_df['accounts_payable'] / raw_df['operate_cost'] * 365 / raw_df['quarter'] * 4
raw_df['fixed_asset_turnover'] = raw_df['total_revenue'] / raw_df['fixed_asset']

# liquidity ratios
raw_df['cash_conversion_cycle'] = raw_df[
    'days_sales_receivables'] + raw_df['days_inventory_on_hands'] - raw_df['days_sales_payables']
# solvency ratios
# profitability ratios
raw_df['return_on_asset'] = raw_df['parent_netprofit'] / raw_df['total_assets']

raw_df.head()

,security_code,security_name_abbr,report_date,total_assets,fixed_asset,monetaryfunds,monetaryfunds_yoy,accounts_rece,accounts_rece_yoy,inventory,inventory_yoy,total_liabilities,accounts_payable,accounts_payable_yoy,advance_receivables,advance_receivables_yoy,total_equity,total_equity_yoy,total_assets_yoy,total_liab_yoy,current_ratio,debt_asset_ratio,parent_netprofit,total_revenue,total_operate_cost,total_expense_yoy,toi_ratio,operate_cost,operate_expense,operate_expense_yoy,sale_expense,manage_expense,finance_expense,operate_profit,total_profit,income_tax,operate_income,interest_ni,interest_ni_ratio,netcash_operate,netcash_operate_yoy,sales_services_cash,sales_services_cash_yoy,pay_staff_cash,pay_staff_cash_yoy,netcash_invest,netcash_invest_yoy,receive_invest_income,receive_invest_income_yoy,construct_long_asset,construct_long_asset_yoy,netcash_finance,netcash_finance_yoy,basic_eps,deduct_basic_eps,weightavg_roe,operate_income_yoy,netprofit_yoy,book_value_per_share,netcash_operate_per_share,gross_margin,operate_income_mom,netprofit_mom,qdate,quarter,net_margin,total_asset_turnover,financial_leverage,cash_to_revenue,cfo_to_netprofit,free_cash_flow,days_sales_receivables,days_inventory_on_hands,days_sales_payables,fixed_asset_turnover,cash_conversion_cycle,return_on_asset
1,858,五粮液,2023-09-30 00:00:00,1.518034e+11,5.250182e+09,9.814771e+10,23.727348,1.406224e+08,10.382347,1.562720e+10,4.450182,2.716341e+10,7.453945e+09,-2.070087,21076698.67,27.356450,1.246400e+11,13.631378,12.710345,8.668737,500.658448,17.893812,2.283293e+10,6.253639e+10,3.115269e+10,11.170684,12.113192,1.509677e+10,1.509677e+10,12.642755,6.227271e+09,2.403022e+09,-1.812648e+09,3.168408e+10,3.162806e+10,7.794083e+09,NaN,NaN,NaN,2.238875e+10,462.548766,6.681847e+10,1380.461593,6.169873e+09,127.468819,-2.418850e+09,-49.973149,23638368.0,0.488366,2.443148e+09,50.475141,-1.512960e+10,-312.575618,5.882,NaN,19.20,12.113192,14.24,31.476796,5.767905,75.859228,18.5312,28.9590,2023Q3,3,0.365114,0.549275,5.588524,1.068474,0.706624,6.437533e+10,1.094343,503.765842,240.288986,11.911281,264.571199,0.150411
2,858,五粮液,2023-06-30 00:00:00,1.505734e+11,5.255604e+09,9.255730e+10,22.882954,7.857872e+07,-36.097269,1.609827e+10,12.100769,3.077844e+10,7.615342e+09,16.409137,18726957.90,58.972136,1.197949e+11,13.564457,15.544903,23.958646,441.564437,20.440828,1.703671e+10,4.550638e+10,2.202668e+10,8.798358,10.392431,1.056796e+10,1.056796e+10,11.074424,4.320446e+09,1.756899e+09,-1.250071e+09,2.371557e+10,2.367227e+10,5.879733e+09,NaN,NaN,NaN,1.132982e+10,1377.414072,4.305169e+10,5233.975750,4.794521e+09,582.890232,-1.344449e+09,-163.450321,NaN,NaN,1.345044e+09,163.522637,-1.080791e+10,-1313.963751,4.389,4.364,14.19,10.392431,12.83,29.983544,2.918847,76.776972,-53.8598,-64.1637,2023Q2,2,0.374381,0.604441,4.892170,0.946058,0.477738,4.170664e+10,1.260537,1112.015480,526.042853,8.658640,587.233164,0.113146
4,858,五粮液,2023-03-31 00:00:00,1.604211e+11,5.254760e+09,1.011283e+11,28.742068,8.845112e+07,-50.377978,1.606515e+10,16.024157,3.062763e+10,7.492576e+09,19.232651,16420891.54,38.333959,1.297935e+11,15.119885,16.412182,22.226754,476.174727,19.092019,1.254209e+10,3.113886e+10,1.385794e+10,11.752605,13.034273,6.727661e+09,6.727661e+09,13.104688,2.112310e+09,1.097441e+09,-6.144575e+08,1.747222e+10,1.742402e+10,4.317548e+09,NaN,NaN,NaN,9.535618e+09,114.626692,2.863684e+10,344.240571,3.223645e+09,38.751107,-1.104264e+09,-13.274243,NaN,NaN,1.104204e+09,13.273524,-1.125082e+08,-1.352450,3.231,NaN,10.43,13.034273,15.89,32.607616,2.456615,78.394648,71.1966,87.1583,2023Q1,1,0.402779,0.776428,5.237791,0.919650,0.545759,2.753264e+10,4.147185,3486.371565,1625.997701,5.925839,1864.521050,0.078182
0,858,五粮液,2022-12-31 00:00:00,1.528119e+11,5.312971e+09,9.235843e+10,12.172654,3.568694e+07,-44.406902,1.598066e+10,14.024834,3.612490e+10,7.246803e+09,34.111601,16160671.49,47.311796,1.166870e+11,15.085176,12.604198,5.263412,384.700951,23.640104,2.669066e+10,7.396864e+10,3.704902e+10,9.103559,11.719828,1.817843